# C1 - Introduction to Human and Computer Vision
## Week 4

In [1]:
import os
import re
import matplotlib.pyplot as plt
import cv2
import pickle
import numpy as np

# Get project's root directory
BASE_PATH = os.path.join(re.search(r'.+(Team5)', os.getcwd())[0], 'week4')
os.chdir(BASE_PATH)
BASE_PATH

DATA_DIRECTORY = '../data'

In [2]:
# Read pickle file to see detailed info of the images augmentation
with open(f'{DATA_DIRECTORY}/qsd1_w4/augmentations.pkl', 'rb') as f:
    augmentations_info = pickle.load(f)

# Read pickle file with correspondences
with open(f'{DATA_DIRECTORY}/qsd1_w4/gt_corresps.pkl', 'rb') as f:
    ground_truth = pickle.load(f)

### a) Remove background, detect noise (and filter it)

In [3]:
from src.background_removal import background_removal
from src.noise_removal import denoise_image
from tqdm import tqdm

# Image names
QSD1_w4_names = [f for f in os.listdir(f'{DATA_DIRECTORY}/qsd1_w4/') if f.endswith('.jpg')]
QSD1_w4_names.sort()

# Initialize datasets
BBDD = []
QSD1_w4 = []
QSD1_w4_filtered = []
QSD1_w4_nonAugmented = []

# Load datasets (+ filter)
for image_name in tqdm(QSD1_w4_names):
    # Read QSD1_w4
    image_qsd1 = cv2.imread(f'{DATA_DIRECTORY}/qsd1_w4/{image_name}')
    QSD1_w4.append(image_qsd1)

    # Read non-augmented image
    image_nonAugmented = cv2.imread(f'{DATA_DIRECTORY}/qsd1_w4/non_augmented/{image_name}')
    image_nonAug_bckg_remov = background_removal(image_nonAugmented)  # Remove background in non-augmented image
   
    QSD1_w4_nonAugmented.append(image_nonAug_bckg_remov)

    # Filter image from QSD1_w4
    filtered_image = background_removal(denoise_image(image_qsd1))  # Detect noise (and clean it) + Remove background
    QSD1_w4_filtered.append(filtered_image)


# Read BBDD
BBDD_names = [f for f in os.listdir(f'{DATA_DIRECTORY}/BBDD/') if f.endswith('.jpg')]
BBDD_names.sort()

for image_name in tqdm(BBDD_names):
    image_bbdd = cv2.imread(f'{DATA_DIRECTORY}/BBDD/{image_name}')
    BBDD.append(image_bbdd)

100%|██████████| 287/287 [00:03<00:00, 89.08it/s] 


In [ ]:
# Plot some examples
img_number = 17


fig, axes = plt.subplots(1, 3)  # 1 fila, 3 columnas

# QSD1_w4 image
image = QSD1_w4[img_number]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
axes[0].imshow(image)
axes[0].set_title('QSD1_w4 image')
axes[0].axis('Off')

# Filtered image (background removal + denoise)
image = QSD1_w4_filtered[img_number][0]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
axes[1].imshow(image)
axes[1].set_title('Filtered image')
axes[1].axis('Off')

# Non-augmented image
image = QSD1_w4_nonAugmented[img_number][0]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
axes[2].imshow(image)
axes[2].set_title('Non-augmented image')
axes[2].axis('Off')

plt.show()
plt.close()

### b) Detect keypoints and compute descriptors

In [ ]:
'''
flags = {
  DEFAULT = 0,
  DRAW_OVER_OUTIMG = 1,
  NOT_DRAW_SINGLE_POINTS = 2,
  DRAW_RICH_KEYPOINTS = 4
}
'''
def draw_keypoints(image, kp, flags=0):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img2 = cv2.drawKeypoints(gray_image, kp, None,(255,0,0),flags=flags)
    plt.imshow(img2) 

#### SIFT

In [ ]:
def sift_descriptor(image, params={}):
    '''
    Compute SIFT descriptors for a given image
    :param image: image to compute the descriptors
    :param params: parameters for the SIFT algorithm
    :return: keypoints and descriptors

    default params = {
        'nfeatures': 0,
        'nOctaveLayers': 3,
        'contrastThreshold': 0.04,
        'edgeThreshold': 10,
        'sigma': 1.6
    }
    '''
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(**params)
    kp, des = sift.detectAndCompute(img_gray, None)

    return (kp, des)

In [ ]:

params = {
    'nfeatures': 1500,
    'nOctaveLayers': 3,
}

sift_query = []
for picture in tqdm(QSD1_w4_filtered):
    res = []
    for painting in picture:
        kp, des = sift_descriptor(painting, params=params)
        res.append({'kp': kp, 'des': des})
    sift_query.append(res)


sift_bd = []
for painting in tqdm(BBDD):
    kp, des = sift_descriptor(painting, params=params)
    sift_bd.append({'kp': kp, 'des': des})

In [ ]:
idx1 = 16 # Query image
idx2 = 11 # BBDD image

des1 = sift_query[idx1][0]['des']
des2 = sift_bd[idx2]['des']

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
# Match descriptors.
matches = bf.match(des1,des2)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
img3 = cv2.drawMatches(
    QSD1_w4_filtered[idx1][0], sift_query[idx1][0]['kp'],
    BBDD[idx2], sift_bd[idx2]['kp'],
    matches[:20], None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
print('Distance:', np.mean([m.distance for m in matches]))
plt.imshow(img3), plt.axis('off'), plt.show()

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

res = []
dist_res = []
for i in tqdm(range(30)):
    picture_res = []
    picture_distances = []
    for j in range(0, len(sift_query[i])):
        distances = []
        for k in range(0, len(sift_bd)):
            if sift_bd[k]['des'] is not None:
                matches = bf.match(sift_query[i][j]['des'], sift_bd[k]['des'])
                matches = sorted(matches, key = lambda x:x.distance)
                # Take top 20 matches
                distance = np.mean([match.distance for match in matches[:20]])
                distances.append(distance)
            else:
                distances.append(10000)

        # Take top 10 matches
        most_similar = np.argsort(distances)[:10]
        most_similar_distances = [distances[idx] for idx in most_similar]
        if most_similar_distances[0] == 10000:
            most_similar = [-1]
            most_similar_distances = [10000]
        if (
            most_similar_distances[1] - most_similar_distances[0] < 
            2*np.mean(np.array(most_similar_distances[2:10]) - np.array(most_similar_distances[1:9]))
        ):
            most_similar = [-1]
        picture_res.append(most_similar)
        picture_distances.append(most_similar_distances)
    
    res.append(picture_res)
    dist_res.append(picture_distances)


In [ ]:
image = QSD1_w4_filtered[24][0]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
print(res)

In [ ]:
import importlib
import src.metrics  # re-import the module to make sure it's recognized
importlib.reload(src.metrics)

from src.metrics import mapk

# Now you can call the updated mapk function
mapk(ground_truth, res, k=1)


#### HOG

In [4]:
from skimage.feature import hog
from skimage import exposure
from scipy.spatial.distance import euclidean

params={
    'shape': (256,256),  # Shape we want to resize the image to
    'pixels_per_cell': (8,8),
    'cells_per_block': (2,2),
}

def hog_descriptor(image, shape: tuple, pixels_per_cell: tuple, cells_per_block: tuple):
    
    # Image to grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize image (we need same dimensions in all the image descriptros to compare)
    image = cv2.resize(image, shape)

    # Compute HOG
    hog_descriptor, hog_image = hog(image, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)
    
    # Rescale intensity, otherwise we may see everyhing black
    hog_image = exposure.rescale_intensity(hog_image, in_range=(0, np.max(hog_image)/10))

    return hog_descriptor, hog_image


In [5]:
hog_query = []
original_query = [ ]  # Imatges del QSD1_W4 una vegada hem aplicat el background removal i en cas que hi hagi dos quadres els separem en dues imatges diferents
for picture in tqdm(QSD1_w4_filtered):
    res = []
    for painting in picture: 
        original_query.append(painting)      
        des, image = hog_descriptor(painting, **params)
        res.append({'descriptor': des, 'image': image})
    hog_query.append(res)

hog_bd = []
for painting in tqdm(BBDD):    
    des, image = hog_descriptor(painting, **params)
    hog_bd.append({'descriptor': des, 'image': image})


100%|██████████| 287/287 [00:14<00:00, 19.82it/s]


In [ ]:
k = 3

res = []
distances_res = []
for i in tqdm(range(30)):
    picture_res = []
    picture_distances = []
    
    # For each painting inside an image from hog_query[i]
    for j in range(len(hog_query[i])):
        distances = []

        query_descriptor = hog_query[i][j]['descriptor']
        for p in range(len(hog_bd)): 
            bd_descriptor = hog_bd[p]['descriptor']
            distance = euclidean(query_descriptor, bd_descriptor)
            distances.append((distance, p))

        distances.sort(key=lambda x: x[0])

        nearest_neighbors = [idx for _, idx in distances[:k]] 
        nearest_distances = [round(dist, 2) for dist, _ in distances[:k]]

        picture_res.append(nearest_neighbors)
        picture_distances.append(nearest_distances)

    res.append(picture_res)
    distances_res.append(picture_distances)
    
print(res)
print(ground_truth)
print(distances_res)

100%|██████████| 30/30 [00:00<00:00, 41.92it/s]

[[[40, 282, 11]], [], [[48, 254, 238], [251, 282, 11]], [[32, 11, 282]], [[161, 11, 40]], [[81, 40, 11]], [[54, 238, 202], [38, 182, 229]], [[282, 40, 32]], [[128, 54, 238]], [[155, 11, 282], [258, 215, 57]], [[136, 137, 54], [21, 92, 54]], [[78, 21, 74]], [[32, 11, 282]], [[53, 238, 54]], [[93, 197, 198]], [[12, 21, 74]], [[282, 11, 245], [232, 282, 32]], [], [[233, 282, 51], [215, 57, 259]], [[93, 198, 78]], [[272, 282, 32], [117, 88, 32]], [[54, 238, 11]], [[242, 176, 221]], [[260, 163, 40]], [[94, 215, 232], [132, 104, 215]], [[57, 9, 174]], [[78, 74, 221]], [[127, 282, 32]], [[47, 229, 52], [13, 115, 225]], [[11, 282, 32]]]
[[-1], [150], [48, 251], [32], [161], [81], [62, 38], [-1], [128], [155, 258], [136, 76], [-1], [-1], [53], [-1], [12], [11, 280], [-1], [182, 252], [-1], [272, 117], [-1], [242], [260], [94, 132], [223], [-1], [127], [47, 13], [-1]]
[[[19.37, 19.92, 20.29]], [], [[21.66, 22.85, 22.87], [16.22, 21.29, 21.83]], [[17.7, 19.69, 20.2]], [[14.42, 17.94, 18.41]], [[1

In [24]:
import importlib
import src.metrics 
importlib.reload(src.metrics)

from src.metrics import mapk

mapk(ground_truth, res, k=1)

-1 [40, 282, 11]
48 [48, 254, 238]
251 [251, 282, 11]
32 [32, 11, 282]
161 [161, 11, 40]
81 [81, 40, 11]
62 [54, 238, 202]
38 [38, 182, 229]
-1 [282, 40, 32]
128 [128, 54, 238]
155 [155, 11, 282]
258 [258, 215, 57]
136 [136, 137, 54]
76 [21, 92, 54]
-1 [78, 21, 74]
-1 [32, 11, 282]
53 [53, 238, 54]
-1 [93, 197, 198]
12 [12, 21, 74]
11 [282, 11, 245]
280 [232, 282, 32]
182 [233, 282, 51]
252 [215, 57, 259]
-1 [93, 198, 78]
272 [272, 282, 32]
117 [117, 88, 32]
-1 [54, 238, 11]
242 [242, 176, 221]
260 [260, 163, 40]
94 [94, 215, 232]
132 [132, 104, 215]
223 [57, 9, 174]
-1 [78, 74, 221]
127 [127, 282, 32]
47 [47, 229, 52]
13 [13, 115, 225]
-1 [11, 282, 32]


0.5675675675675675

#### DAISY

In [ ]:
from skimage.feature import daisy

def daisy_descriptor(image, step=250, radius=50, rings=3, histograms=6, orientations=8):
    # Convertimos la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculamos los descriptores DAISY sin visualización
    descriptors, dsc_image = daisy(
        gray_image,
        step=step,
        radius=radius,
        rings=rings,
        histograms=histograms,
        orientations=orientations,
        visualize=True
    )
    
    return descriptors, dsc_image


In [ ]:
def daisy_descriptor(image, step=158, radius=50, rings=3, histograms=6, orientations=8):
    # Convertimos la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Usamos un detector de keypoints (aquí FAST, pero puedes usar otros)
    fast = cv2.FastFeatureDetector_create()
    keypoints = fast.detect(gray_image, None)
    
    # Calculamos los descriptores DAISY en los puntos clave detectados
    descriptors = daisy(
        gray_image,
        step=step,
        radius=radius,
        rings=rings,
        histograms=histograms,
        orientations=orientations,
        visualize=False
    )
    
    # Alineamos los descriptores con los keypoints (si es necesario)
    keypoints_daisy = []
    keypoint_descriptors = []
    
    for kp in keypoints:
        # Para cada keypoint, calculamos su posición (en coordenadas de píxeles)
        y, x = kp.pt
        x, y = int(x), int(y)
        
        # Aquí usamos una verificación simple, que si el keypoint está dentro de la imagen
        if x < descriptors.shape[1] and y < descriptors.shape[0]:
            keypoints_daisy.append(kp)
            keypoint_descriptors.append(descriptors[y, x, :])  # Tomamos el descriptor en esa ubicación
    
    # Convertimos las listas a arrays de numpy para facilitar el uso
    keypoints_daisy = np.array(keypoints_daisy)
    keypoint_descriptors = np.array(keypoint_descriptors)
    
    return keypoints_daisy, keypoint_descriptors

In [ ]:
daisy_query = []
for picture in tqdm(QSD1_w4_filtered[:4]):
    for painting in picture:
        kp, des = daisy_descriptor(painting)
        daisy_query.append({'kp': kp, 'des': des})


daisy_bd = []
for picture in tqdm(QSD1_w4_nonAugmented[:4]):
    for painting in picture:
        kp, des = daisy_descriptor(painting)
        daisy_bd.append({'kp': kp, 'des': des})

In [ ]:
# Mostrar la imagen original y la visualización del descriptor DAISY
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Imagen original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(dsc_image, cmap='gray')
plt.title('Descriptor DAISY')
plt.axis('off')

plt.show()

# 2
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image1, cmap='gray')
plt.title('Imagen original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(dsc_image1, cmap='gray')
plt.title('Descriptor DAISY')
plt.axis('off')

plt.show()

In [ ]:
# Convertir descriptores a tipo float32 para el matcher de OpenCV
descriptors = des_daisy.astype(np.float32).reshape(-1, 152)
descriptors1 = des_daisy1.astype(np.float32).reshape(-1, 152)

# Crear el matcher Brute-Force con la métrica de distancia Euclidiana
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Encontrar las mejores coincidencias
matches = bf.match(descriptors, descriptors1)

# Ordenar las coincidencias por distancia (las más cercanas son las mejores)
matches = sorted(matches, key=lambda x: x.distance)

# Para dibujar los keypoints necesitamos crear un arreglo de KeyPoint
# Generar keypoints para la primera y segunda imagen a partir de la malla DAISY
keypoints = []
keypoints1 = []

step = 158  # El valor del 'step' usado en DAISY
radius = 50  # El radio del descriptor

# Generar keypoints para la imagen original
for i in range(des_daisy.shape[0]):
    for j in range(des_daisy.shape[1]):
        keypoints.append(cv2.KeyPoint(j * step + radius, i * step + radius, 1))

# Generar keypoints para la segunda imagen
for i in range(des_daisy1.shape[0]):
    for j in range(des_daisy1.shape[1]):
        keypoints1.append(cv2.KeyPoint(j * step + radius, i * step + radius, 1))

# Dibujar los matches entre las imágenes
img_matches = cv2.drawMatches(
    image, keypoints,  # Imagen original y sus keypoints
    image1, keypoints1,  # Imagen de referencia y sus keypoints
    matches[:30], None
)

# Mostrar el resultado con matplotlib
plt.figure(figsize=(15, 10))
plt.imshow(img_matches)
plt.title('Coincidencias entre imagen 1 y imagen 2 usando DAISY')
plt.axis('off')  # Desactivar los ejes para que sea más limpio
plt.show()



In [ ]:
surf_query = []
for picture in tqdm(QSD1_w4_filtered):
    for painting in picture:
        kp, des = surf_descriptor(painting)
        surf_query.append({'kp': kp, 'des': des})


surf_bd = []
for picture in tqdm(QSD1_w4_nonAugmented):
    for painting in picture:
        kp, des = surf_descriptor(painting)
        surf_bd.append({'kp': kp, 'des': des})

#### ORB

In [ ]:
def orb_descriptor(image, params={}):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(**params)
    kp, des = orb.detectAndCompute(img_gray, None)

    return (kp, des)

# Task 2

### a) Find tentative matches based on similarity of local appearance and verify matches 

### b) Implement a system to discard queries not in the data set (unknowns)

# Task 3

### a) Evaluate system based on keypoint descriptors on QSD1-W4

### b) Compare your best query system from previous week on QSD1-W4

# Task 4

### a) Create pkl file